In [1]:
import pandas as pd
import numpy as np
from numpy import inf, mean, median
import math
import datetime
import json
import os
import glob
import pickle
import requests
# import requests_cache
import plotly.graph_objects as go
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, HeatMap
import geopandas as gpd
import osm2geojson
import geojson
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString

# Load Data

## нерабочая хрень

In [3]:
data_spb_token = 'b84fc5afb5fe86c84f51111ada361463937f3100'

In [18]:
data_spb_dataset_id = 7830001067 # -The_subway_entrances'

In [23]:
data_spb_request = f'https://data.gov.spb.ru/api/v1/datasets/'
data_spb_request

'https://data.gov.spb.ru/api/v1/datasets/'

In [24]:
data_spb_get = requests.get(data_spb_request, headers={'Authorization': f'Token {data_spb_token}'})
data_spb_get

<Response [200]>

In [27]:
data_spb_datasets = data_spb_get.json()

In [28]:
data_spb_datasets

[{'id': 2,
  'name': 'Информация о государственных казенных учреждениях, подведомственных Архивному комитету Санкт-Петербурга'},
 {'id': 3,
  'name': 'Статистика о количестве фондов и описей в архивах Санкт-Петербурга'},
 {'id': 4,
  'name': 'Объекты культурного наследия на территории Санкт-Петербурга'},
 {'id': 5,
  'name': 'Наименования органов записи актов гражданского состояния Санкт-Петербурга'},
 {'id': 6,
  'name': 'Перечень ресурсоснабжающих организаций - владельцев сетей инженерно-технического обеспечения и электрических сетей в Санкт-Петербурге'},
 {'id': 7, 'name': 'Перечень государственных программ Санкт-Петербурга'},
 {'id': 8,
  'name': 'Реестр специалистов в области ветеринарии, занимающихся предпринимательской деятельностью  на территории  Санкт-Петербурга'},
 {'id': 9, 'name': 'Данные о согласованных адресных программах'},
 {'id': 10,
  'name': 'Данные об ограничении движения транспорта на период производства работ'},
 {'id': 11, 'name': 'Данные о действующих ордерах н

In [29]:
data_spb_dataset_id = 185

In [30]:
data_spb_request = f'https://data.gov.spb.ru/api/v1/datasets/{data_spb_dataset_id}/versions/latest/data/'
data_spb_request

'https://data.gov.spb.ru/api/v1/datasets/185/versions/latest/data/'

In [31]:
data_spb_get = requests.get(data_spb_request, headers={'Authorization': f'Token {data_spb_token}'})
data_spb_get

<Response [200]>

In [32]:
data_spb_metro_entrances = data_spb_get.json()

In [33]:
data_spb_metro_entrances

[{'num_id': 1,
  'row': {'note': '-',
   'number': '1',
   'name': 'пр.Ветеранов-1',
   'address': 'г. Санкт-Петербург, бульвар Новаторов, д.98 в'}},
 {'num_id': 2,
  'row': {'note': '-',
   'number': '2',
   'name': 'пр.Ветеранов-2',
   'address': 'г. Санкт-Петербург, бульвар Новаторов, д.108 а'}},
 {'num_id': 3,
  'row': {'note': '-',
   'number': '3',
   'name': 'Ленинский пр.-1',
   'address': 'г. Санкт-Петербург, Ленинский пр., д.127'}},
 {'num_id': 4,
  'row': {'note': '-',
   'number': '4',
   'name': 'Ленинский пр.-2',
   'address': 'г. Санкт-Петербург, бульвар Новаторов, д.10'}},
 {'num_id': 5,
  'row': {'note': '-',
   'number': '5',
   'name': 'Автово',
   'address': 'г. Санкт-Петербург, пр.Стачек, д.90 корпус 2'}},
 {'num_id': 6,
  'row': {'note': '-',
   'number': '6',
   'name': 'Кировский з-д',
   'address': 'г. Санкт-Петербург, пр.Стачек, д.70'}},
 {'num_id': 7,
  'row': {'note': '-',
   'number': '7',
   'name': 'Нарвская',
   'address': 'г. Санкт-Петербург, пл. Стачек

## end нерабочая хрень

In [35]:
data_spb_metro_entrances = pd.read_csv('data_spb_metro_entrances.csv')

In [36]:
data_spb_metro_entrances

,number,name,address,note
0,1,пр.Ветеранов-1,"г. Санкт-Петербург, бульвар Новаторов, д.98 в",-
1,2,пр.Ветеранов-2,"г. Санкт-Петербург, бульвар Новаторов, д.108 а",-
2,3,Ленинский пр.-1,"г. Санкт-Петербург, Ленинский пр., д.127",-
3,4,Ленинский пр.-2,"г. Санкт-Петербург, бульвар Новаторов, д.10",-
4,5,Автово,"г. Санкт-Петербург, пр.Стачек, д.90 корпус 2",-
...,...,...,...,...
75,76,Спортивная-2,"г. Санкт-Петербург, набережная Макарова",-
76,77,Чкаловская,"г. Санкт-Петербург, ул. Большая Зеленина, д. 12",-
77,78,Крестовский остров,"г. Санкт-Петербург, Рюхина улица, д.8",-
78,79,Старая деревня,"г. Санкт-Петербург, Торфяная дорога, д.4",-


# Coordinates

In [37]:
def address_to_geo_coord_api(address):
    r = requests.get(f'http://search.maps.sputnik.ru/search/addr?q={address}')
    response = r.json()
    coordinates = response['result']['address'][0]['features'][0]['geometry']['geometries'][0]['coordinates']
    coordinates = coordinates[::-1]
    return coordinates

In [38]:
coordinates = []
for address in data_spb_metro_entrances['address']:
    coord = address_to_geo_coord_api(address)
    coordinates.append(coord)

In [39]:
data_spb_metro_entrances['coordinates'] = coordinates

In [40]:
data_spb_metro_entrances['lat'], data_spb_metro_entrances['lon'] = zip(*data_spb_metro_entrances['coordinates'])

In [41]:
data_spb_metro_entrances.head()

,number,name,address,note,coordinates,lat,lon
0,1,пр.Ветеранов-1,"г. Санкт-Петербург, бульвар Новаторов, д.98 в",-,"[59.84101, 30.253126]",59.841010,30.253126
1,2,пр.Ветеранов-2,"г. Санкт-Петербург, бульвар Новаторов, д.108 а",-,"[59.841515, 30.250565]",59.841515,30.250565
2,3,Ленинский пр.-1,"г. Санкт-Петербург, Ленинский пр., д.127",-,"[59.85131, 30.265657]",59.851310,30.265657
3,4,Ленинский пр.-2,"г. Санкт-Петербург, бульвар Новаторов, д.10",-,"[59.849537, 30.269468]",59.849537,30.269468
4,5,Автово,"г. Санкт-Петербург, пр.Стачек, д.90 корпус 2",-,"[59.867313, 30.261372]",59.867313,30.261372


In [42]:
data_spb_metro_entrances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   number       80 non-null     int64  
 1   name         80 non-null     object 
 2   address      80 non-null     object 
 3   note         80 non-null     object 
 4   coordinates  80 non-null     object 
 5   lat          80 non-null     float64
 6   lon          80 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.5+ KB


In [43]:
data_spb_metro_entrances.to_csv('data_spb_metro_entrances_upd.csv', sep=';', index=False)

In [47]:
map_spb_metro_entrances = folium.Map(width=1200, height=700, zoom_start=10, location=[59.93, 30.30])

for index, row in data_spb_metro_entrances.iterrows():
    folium.CircleMarker(location=row['coordinates'],
                        radius = 2,
                        popup=row['name'] + ' ' + row['address'],
                        fill_color='blue',
                        color=None,
                        fill_opacity = 1
                       ).add_to(map_spb_metro_entrances)

map_spb_metro_entrances

In [48]:
def f_list_to_point(p):
    return Point(tuple(p))

In [49]:
data_spb_metro_entrances['geometry'] = data_spb_metro_entrances['coordinates'].apply(f_list_to_point)

In [50]:
data_spb_metro_entrances_gdf = gpd.GeoDataFrame(data_spb_metro_entrances, geometry='geometry')

In [51]:
data_spb_metro_entrances_gdf.head()

,number,name,address,note,coordinates,lat,lon,geometry
0,1,пр.Ветеранов-1,"г. Санкт-Петербург, бульвар Новаторов, д.98 в",-,"[59.84101, 30.253126]",59.841010,30.253126,POINT (59.84101 30.25313)
1,2,пр.Ветеранов-2,"г. Санкт-Петербург, бульвар Новаторов, д.108 а",-,"[59.841515, 30.250565]",59.841515,30.250565,POINT (59.84152 30.25057)
2,3,Ленинский пр.-1,"г. Санкт-Петербург, Ленинский пр., д.127",-,"[59.85131, 30.265657]",59.851310,30.265657,POINT (59.85131 30.26566)
3,4,Ленинский пр.-2,"г. Санкт-Петербург, бульвар Новаторов, д.10",-,"[59.849537, 30.269468]",59.849537,30.269468,POINT (59.84954 30.26947)
4,5,Автово,"г. Санкт-Петербург, пр.Стачек, д.90 корпус 2",-,"[59.867313, 30.261372]",59.867313,30.261372,POINT (59.86731 30.26137)


In [52]:
data_spb_metro_entrances_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   number       80 non-null     int64   
 1   name         80 non-null     object  
 2   address      80 non-null     object  
 3   note         80 non-null     object  
 4   coordinates  80 non-null     object  
 5   lat          80 non-null     float64 
 6   lon          80 non-null     float64 
 7   geometry     80 non-null     geometry
dtypes: float64(2), geometry(1), int64(1), object(4)
memory usage: 5.1+ KB


In [53]:
with open('data_spb_metro_entrances_gdf.pickle', 'wb') as _f:
    pickle.dump(data_spb_metro_entrances_gdf, _f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data_spb_metro_entrances_gdf.pickle', 'rb') as _f:
    data_spb_metro_entrances_gdf = pickle.load(_f)